## RAG로 AI소믈리에 wine pairing

In [11]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

In [2]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI

### LLM을 통한 요리 정보 해석
- 이미지 (image to text)
- 입력 : url
- 출력 : 요리명, 설명
- 함수로 정의, runnableLambda 객체 사용하기

In [21]:
def desc_dish_flavor(input_data):

    prompt = ChatPromptTemplate([
        ("system" , """
        # Role
    당신은 세계적인 수준의 '디지털 소믈리에'입니다. 사용자가 제공한 음식 사진을 시각적으로 분석하여, 해당 요리의 특징을 도출하고 최적의 와인 페어링을 제안하는 것이 당신의 임무입니다.
    
    # Analysis Process (사고 단계)
    1. **시각적 요소 식별**: 주재료(단백질 종류), 조리 방식(굽기, 찌기, 튀기기), 소스의 색상 및 질감, 곁들임 채소(가니쉬)를 파악합니다.
    2. **맛 프로파일 유추**: 시각 정보를 바탕으로 예상되는 [염도, 산도, 당도, 지방함량, 질감, 향신료 강도]를 수치화하거나 상세히 묘사합니다.
    3. **페어링 전략 설정**: '보완(Congruent)' 또는 '대비(Contrast)' 전략 중 무엇을 사용할지 결정합니다.
    4. **와인 그룹 선정**: 구체적인 품종이나 지역을 포함한 최적의 와인 그룹을 추천합니다.
    
    # Output Format (응답 형식)
    1. **요리 분석**: (예: 노릇하게 구워진 마이야르 반응이 특징인 소고기 스테이크, 진한 레드와인 소스)
    2. **예상되는 맛**: (예: 높은 단백질 농도, 버터의 풍미, 후추의 매콤함)
    3. **추천 와인 카테고리**: (예: 풀 바디 레드 와인)
    4. **추천 품종 및 지역**: (예: 프랑스 보르도의 카베르네 소비뇽 또는 호주의 쉬라즈)
    5. **페어링 이유**: (전문 소믈리에의 언어로 설명)
    6. **서빙 팁**: (적정 온도 및 디캔팅 여부)
    """),
        HumanMessagePromptTemplate.from_template([
            {"text": "이 음식에 맞는 와인을 찾을 수 있도록 이 음식을 분석해 주세요"},
            {"image_url": "{image_url}"} # image_url는 정해줘 있음.
        ])
    ])
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                        temperature=0.2,
                        google_api_key = GEMINI_API_KEY)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser

    return chain    
    

In [26]:
from langchain_core.runnables import RunnableLambda

r1 = RunnableLambda(desc_dish_flavor)
input_data = { "image_url" :
    "https://upload3.inven.co.kr/upload/2025/12/31/bbs/i1701205734.jpg"}

response = r1.invoke(input_data)

In [27]:
from IPython.display import display, Markdown


display(Markdown(response))

훌륭한 돼지고기 구이 사진입니다! 제가 분석하여 최적의 와인 페어링을 제안해 드리겠습니다.

---

1.  **요리 분석**:
    뼈가 붙은 삼겹살을 노릇하게 구워낸 요리로, 겉은 바삭한 마이야르 반응이 선명하고 속은 촉촉해 보이는 돼지고기 구이입니다. 고명으로 깨와 다진 파(추정)가 올라가 있어, 과도하게 달거나 짜지 않은 담백한 양념의 구이임을 시사합니다.

2.  **예상되는 맛**:
    삼겹살 특유의 높은 지방 함량에서 오는 풍부한 고소함과 감칠맛이 도드라질 것입니다. 구운 돼지고기의 육향과 은은한 캐러멜화된 풍미가 특징이며, 겉은 바삭하고 속은 부드러운 질감이 예상됩니다. 강한 향신료보다는 고기 본연의 맛을 살린 간이 주를 이룰 것으로 보입니다.

3.  **추천 와인 카테고리**:
    미디엄-풀 바디 레드 와인 (Medium-Full Bodied Red Wine)

4.  **추천 품종 및 지역**:
    *   **쉬라즈(Syrah/Shiraz)**: 호주(Shiraz) 또는 프랑스 론 밸리(Syrah)
    *   **템프라니요(Tempranillo)**: 스페인 리오하(Rioja)

5.  **페어링 이유**:
    이 삼겹살 구이는 풍부한 지방과 고소한 풍미가 핵심이므로, 와인의 **적절한 산도와 탄닌이 지방의 느끼함을 깔끔하게 씻어주고, 와인의 농축된 과일향과 스파이시함이 구이의 고소하고 진한 맛을 보완하고 증폭시키는 보완(Congruent) 및 대비(Contrast) 전략**을 활용합니다.

    *   **쉬라즈/시라**: 짙은 과일향(블랙베리, 플럼)과 후추 같은 스파이시함, 그리고 묵직한 바디감을 가진 쉬라즈/시라는 삼겹살의 기름진 맛을 견고한 구조감으로 잡아줍니다. 구운 고기의 스모키한 뉘앙스와 와인의 오크 숙성에서 오는 복합적인 향이 훌륭하게 어우러져, 서로의 풍미를 더욱 깊게 만들어줍니다.
    *   **템프라니요**: 특히 오크 숙성된 스페인 리오하의 템프라니요는 체리, 자두 등의 붉은 과일향과 함께 가죽, 담배, 바닐라 같은 복합적인 아로마를 발산합니다. 부드러우면서도 존재감 있는 탄닌과 적절한 산미는 삼겹살의 지방을 효과적으로 중화시키고, 숙성 와인의 깊이감이 구운 고기의 감칠맛을 한층 더 끌어올려줍니다.

6.  **서빙 팁**:
    *   **적정 온도**: 두 품종 모두 16~18°C 정도로 서빙하는 것이 좋습니다. 너무 차가우면 와인의 풍미가 닫히고, 너무 따뜻하면 알코올이 도드라질 수 있습니다.
    *   **디캔팅 여부**: 영한 쉬라즈나 리오하 크리안자(Crianza)급 템프라니요라면 30분 정도의 디캔팅으로 와인의 향을 열어주는 것이 좋습니다. 리오하 레제르바(Reserva) 또는 그란 레제르바(Gran Reserva)와 같은 장기 숙성 템프라니요는 1시간 이상 디캔팅하여 복합적인 아로마와 침전물을 걸러내면 더욱 깊은 맛을 즐길 수 있습니다.

In [12]:
# llm = ChatOpenAI(model="gpt-4o-mini", 
#                  temperature=0.3, 
#                  openai_api_key=OPENAI_API_KEY)

# output_parser = StrOutputParser()

In [13]:
# from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
#                         temperature=0.2,
#                         google_api_key = GEMINI_API_KEY)

In [14]:
# # LCEL (Langchain Expression Language)
# chain = prompt | llm | output_parser

In [15]:
# response = chain.invoke({
#                         "image_url":
#                          "https://www.elle.co.kr/resources/online/online_image/2024/10/17/59555f71-b052-490b-a662-059788a56b83.png"}
#                        )

In [25]:
# from IPython.display import display, Markdown


# display(Markdown(response))